In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
from LinearMPC.MPCVelControl import MPCVelControl
from PIControl.PIControl import PIControl
import numpy as np
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
# Must remove "raise ValueError("State or input constraints violated")" in rocket.py l.423 to run
Ts = 0.05
sim_time = 20
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])  # initial state
pos_target = np.array([0, 0, 10.0])

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller = PIControl(pos_target)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, pos_control=pos_controller, method="nonlinear"
)

In [ ]:
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 0.5
vis.animate(
    t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=ref[:, :-1],
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
);

In [ ]:
# Print to know how big the constraint violation is
from LinearMPC.utils import ALPHA, BETA, LB_X, UB_X

print(f"Max alpha: {np.rad2deg(max(np.abs(x_cl[ALPHA])))}")
print(f"Max beta: {np.rad2deg(max(np.abs(x_cl[ALPHA])))}")

eps = 1e-5
for alpha in x_cl[ALPHA]:
    if alpha < LB_X[ALPHA] - eps or alpha > UB_X[ALPHA] + eps:
        print("Alpha constraint violated:", np.rad2deg(alpha))
for beta in x_cl[BETA]:
    if beta < LB_X[BETA] - eps or beta > UB_X[BETA] + eps:
        print("Beta constraint violated:", np.rad2deg(beta))

In [ ]:
from LinearMPC.utils import plot_trajectory
plot_trajectory(t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, mpc.mpc_x)
plot_trajectory(t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, mpc.mpc_y)
plot_trajectory(t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, mpc.mpc_z)
plot_trajectory(t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, mpc.mpc_roll)
